Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
# ## DONT CHANGE THIS CELL 
# # this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

In [ ]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=xxxxxxxxxxx

In [8]:
import os
import pandas
import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

In [31]:
x_test = pandas.read_csv("data/test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
x_test = x_test.drop("position", axis=1)
x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
x_test.insert(3, "word1", word1)
x_test.insert(5, "word2", word2)
x_test.to_csv("data/test.csv", header=None, index=False)

In [32]:
import dill
with open(os.path.join("data", "Field_TEXT"), 'rb') as f:
    TEXT = dill.load(f)

In [33]:
TEXT.vocab.vectors.shape

torch.Size([7462, 100])

In [36]:
fields = [
    ('word', TEXT),
    ('POS', data.Field(use_vocab=False, sequential=False)), 
    ('sen1', TEXT),
    ('word1', data.Field(use_vocab=False, sequential=False)),
    ('sen2', TEXT),
    ('word2', data.Field(use_vocab=False, sequential=False)),
]

test_set = data.TabularDataset.splits(
    path = "data",
    test = "test.csv",
    format = 'csv',
    fields = fields,
    skip_header = False
)

(test_itr,) = data.BucketIterator.splits(
    test_set,
    #sort_key = lambda sample : len(sample.sen1),
    sort = False,
    batch_size = 32,
    repeat = False
)

In [37]:
test_itr

In [55]:
import os
import time
import datetime
import pandas
import logging
from argparse import ArgumentParser
from pdb import set_trace
import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

class myModel(nn.Module):
    def __init__(self, pretrained_embeddings, embed_dim=100, hidden_dim=100, device=torch.device('cpu')):
        super(myModel, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.device = device
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        self.dropout = nn.Dropout(p = 0.5)
        self.bilstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.similarity = nn.CosineSimilarity(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        '''
            [torchtext.legacy.data.batch.Batch of size 32]
                [.word]:[torch.LongTensor of size 1x32]
                [.POS]:[torch.LongTensor of size 32]
                [.sen1]:[torch.LongTensor of size 17x32]
                [.word1]:[torch.LongTensor of size 32]
                [.sen2]:[torch.LongTensor of size 26x32]
                [.word2]:[torch.LongTensor of size 32]
                [.label]:[torch.LongTensor of size 32]
        '''
        input1 = input.sen1.permute(1,0).to(self.device)
        input2 = input.sen2.permute(1,0).to(self.device)

        # embed the tokens (indices in the vocab) 
        out1 = self.dropout(self.embedding(input1))
        out2 = self.dropout(self.embedding(input2))

        out1, (_, _) = self.bilstm(out1)
        out2, (_, _) = self.bilstm(out2)

        # word of interest
        woi1_idx = torch.repeat_interleave(torch.unsqueeze(torch.unsqueeze(input.word1, 1), 2), 2*self.hidden_dim, dim=2).to(self.device)
        woi2_idx = torch.repeat_interleave(torch.unsqueeze(torch.unsqueeze(input.word2, 1), 2), 2*self.hidden_dim, dim=2).to(self.device)

        # gather the word of interest for each sentence in the batch
        out1 = torch.gather(input=out1, dim=1, index=woi1_idx).squeeze()
        out2 = torch.gather(input=out2, dim=1, index=woi2_idx).squeeze()
        
        # compute scores of similarity
        out = self.similarity(out1, out2)
        out = self.sigmoid(out)
        # try:
        #     assert out.shape[0] == self.bs
        # except:
        #     print("AssertionError", out.shape[0], self.bs) # all samples are touched in a epoch

        return out


In [56]:
model = myModel(pretrained_embeddings=TEXT.vocab.vectors, embed_dim=TEXT.vocab.vectors.shape[1], hidden_dim=100, device=torch.device("cpu"))

In [73]:
x = torch.load("data/best-bilstm-params.pt",map_location="cpu")

In [75]:
model.load_state_dict(x['model_dict'])

<All keys matched successfully>

In [45]:
f = open('output.txt', 'r').readlines()
f

['T\n', 'F\n', 'T\n']

In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
    prediciton, gold = prediction.strip(), gold.strip()
    total += 1
    if prediction == gold:
        correct += 1

## Report this as the final validation performance 
print('Performance = '%(correct/total))